In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, roc_auc_score

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

import pickle

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
#a

In [3]:
class uji(): 
    def __init__(self,data): 
        df0 = pd.read_csv('../data3/uji%snorm.csv'%data, delimiter= ",", header=None)
        df0=df0.fillna(0)
        self.com=df0[0]
        self.prot=df0[1282]
        self.x =df0.drop([0,1282], axis=1)
        df0[1300]=0
        
        self.y=df0[1300]
        self.dt=data
class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data3/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['label'], axis=1)
        self.Y = df['label']
        self.dt=data
def dtuji(data): 
    return uji(data) 
def dtlatih(data): 
    return latih(data) 

dtf =dtuji('mbafika') #dtf.com, dtf.prot, dtf.x, dtf.y, dtf.dt
dtl =dtuji('mbalinda') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt
df1= dtlatih('hasilsmote_1') #df1.X,df1.Y,df1.dt
df2= dtlatih('hasilsmote_2') #df2.X,df2.Y,df2.dt
df3= dtlatih('hasilsmote_3') #df3.X,df3.Y,df3.dt
df4= dtlatih('hasilsmote_4') #df4.X,df4.Y,df4.dt
df5= dtlatih('hasilsmote_5') #df5.X,df5.Y,df5.dt

In [4]:
dtg =dtuji('gabung') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt

In [5]:
class uji(): 
    def __init__(self,data): 
        df0 = pd.read_csv('../data3/uji%snorm.csv'%data, delimiter= ",", header=None)
        df0=df0.fillna(0)
        self.com=df0[0]
        self.prot=df0[1282]
        self.x =df0.drop([0,1282], axis=1)
        df0[1300]=0
        
        self.y=df0[1300]
        self.dt=data
class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data3/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['0','1282','label'], axis=1)
        self.Y = df['label']
        self.dt=data
def dtuji(data): 
    return uji(data) 
def dtlatih(data): 
    return latih(data) 
df= dtlatih('latih_parted')

In [6]:
#df.X

In [7]:
scoring = {'AUC': 'roc_auc', 'Accuracy': 'accuracy','F-measure':'f1','Precision':'precision', 'Recall':'recall'}


In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

Using TensorFlow backend.


In [9]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(640, input_dim=1281, init='normal', activation='relu'))
    model.add(Dense(320, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:
def mlp(df):
    mlp = MLPClassifier(max_iter=1000, batch_size=128,hidden_layer_sizes=(640,320,),activation= 'relu',
                        solver= 'adam', alpha= 0.5,learning_rate= 'constant')
    model = Sequential()
    model.add(Dense(640, input_dim=1281, init='uniform', activation='relu'))
    model.add(Dense(320, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_model, nb_epoch=100, batch_size=128, verbose=0,class_weight="balanced")))
    pipeline = Pipeline(estimators)
#    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
#    results = cross_val_score(pipeline, df.X, df.Y, cv=kfold)
#    print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    parameter_space = {
        'hidden_layer_sizes': [(640,320,)],
        'activation': ['relu'],
        'solver': ['adam'],
        'alpha': [0.5],
        'learning_rate': ['constant'],
    }
    mlp = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
    mlp.fit(df.X, df.Y)
    print('done_training')
    filename = '../data3/model/mlp_{0}'.format(df.dt)
    pickle.dump(mlp, open(filename, 'wb'))
    print('done_savemodel')
    return mlp

In [11]:
#rf5_100n_1000m.cv_results_#['mean_test_score']
#stds = clf.cv_results_['std_test_score']
def print_cv_results_mlp(model,o):
    rf = pickle.load(open('../data3/model/{0}_{1}'.format(model.__name__,o.dt), 'rb'))
    print('{0}_{1}'.format(model.__name__,o.dt))
    print('Best parameters found:', rf.best_params_)
    print('AUC :',rf.cv_results_['mean_test_AUC'])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'])
    print('Precision :',rf.cv_results_['mean_test_Precision'])
    print('Recall :',rf.cv_results_['mean_test_Recall'])

In [17]:
mlp1=mlp(df1)
mlp2=mlp(df2)
mlp3=mlp(df3)
mlp4=mlp(df4)
mlp5=mlp(df5)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


done_training
done_savemodel


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


done_training
done_savemodel


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


done_training
done_savemodel


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


done_training
done_savemodel


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


done_training
done_savemodel


In [25]:
print_cv_results_mlp(mlp,df1)
print_cv_results_mlp(mlp,df2)
print_cv_results_mlp(mlp,df3)
print_cv_results_mlp(mlp,df4)
print_cv_results_mlp(mlp,df5)

mlp_hasilsmote_1
Best parameters found: {'activation': 'relu', 'alpha': 0.5, 'hidden_layer_sizes': (640, 320), 'learning_rate': 'constant', 'solver': 'adam'}
AUC : [0.9781747]
Accuracy : [0.93427169]
F-measure : [0.92318014]
Precision : [0.95472489]
Recall : [0.91373967]
mlp_hasilsmote_2
Best parameters found: {'activation': 'relu', 'alpha': 0.5, 'hidden_layer_sizes': (640, 320), 'learning_rate': 'constant', 'solver': 'adam'}
AUC : [0.98021072]
Accuracy : [0.94227789]
F-measure : [0.93394176]
Precision : [0.96146009]
Recall : [0.92122934]
mlp_hasilsmote_3
Best parameters found: {'activation': 'relu', 'alpha': 0.5, 'hidden_layer_sizes': (640, 320), 'learning_rate': 'constant', 'solver': 'adam'}
AUC : [0.97826498]
Accuracy : [0.94266529]
F-measure : [0.93460914]
Precision : [0.95975165]
Recall : [0.92381198]
mlp_hasilsmote_4
Best parameters found: {'activation': 'relu', 'alpha': 0.5, 'hidden_layer_sizes': (640, 320), 'learning_rate': 'constant', 'solver': 'adam'}
AUC : [0.97927911]
Accur

In [14]:
from sklearn import svm

In [15]:
def svm_(df,kern):
    parameter_space0 = {
        'C': [0.03125, 0.5, 8, 128, 2048, 32768],
        'kernel': [kern],#'rbf', 'sigmoid', 'poly','linear'],
        'gamma' : [2, 0.125,  0.0078125,0.00048828125, 0.00003051757],
    }
    classifier0 = svm.SVC()
    classifier0 = GridSearchCV(classifier0, parameter_space0, scoring=scoring,refit='AUC',return_train_score=True, n_jobs=-1, cv=5)
    classifier0.fit(df.X, df.Y)
    print('done_training')
    print('Best parameters found:', classifier0.best_params_)
    filename = '../data3/model/svm_{0}_{1}'.format(kern,df.dt)
    pickle.dump(classifier0, open(filename, 'wb'))
    print('done_savemodel')
    return classifier0

In [12]:
def svm_cg(df,kern,c,g):
    parameter_space0 = {
        'C': [c],
        'kernel': [kern],#'rbf', 'sigmoid', 'poly','linear'],
        'gamma' : [g],
    }
    classifier0 = SVC(probability=True)
    classifier0 = GridSearchCV(classifier0, parameter_space0, scoring=scoring,refit='AUC',return_train_score=True, n_jobs=-1, cv=5)
    classifier0.fit(df.X, df.Y)
    print('done_training')
    print('Best parameters found:', classifier0.best_params_)
    filename = '../data3/model/svm_{0}_{1}'.format(kern,df.dt)
    pickle.dump(classifier0, open(filename, 'wb'))
    print('done_savemodel')
    return classifier0

In [17]:
def save_svm(svm1,df):
    filename = '../data3/model/svm_rbf_{0}'.format(df.dt)
    pickle.dump(svm1, open(filename, 'wb'))

In [97]:
svma=svm_cg(df1,'rbf',0.5, 0.0078125)
svmb=svm_cg(df2,'rbf',8, 0.0078125)
svmc=svm_cg(df3,'rbf', 0.5,  0.0078125)
svmd=svm_cg(df4,'rbf',8, 0.0078125)
svme=svm_cg(df5,'rbf',0.5, 0.0078125)

done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel


D:\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel


In [13]:
svme=svm_cg(df5,'rbf',0.5, 0.0078125)

done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel


In [57]:
save_svm(svm1,df1)
save_svm(svm2,df2)
save_svm(svm3,df3)
save_svm(svm4,df4)
save_svm(svm5,df5)

In [ ]:
svm1=svm_(df1,'rbf')
svm2=svm_(df2,'rbf')
svm3=svm_(df3,'rbf')
svm4=svm_(df4,'rbf')
svm5=svm_(df5,'rbf')

done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel


In [12]:
svm5=svm_(df5,'rbf')

NameError: name 'svm_' is not defined

In [90]:
#rf5_100n_1000m.cv_results_#['mean_test_score']
#stds = clf.cv_results_['std_test_score']
def print_cv_results_svm(o):
    rf = pickle.load(open('../data3/model/svm_rbf_{0}'.format(o.dt), 'rb'))
    print('svm_rbf_{0}'.format(o.dt))
    print('Best parameters found:', rf.best_params_)
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])

In [91]:
print_cv_results_svm(df1)
print_cv_results_svm(df2)
print_cv_results_svm(df3)
print_cv_results_svm(df4)
print_cv_results_svm(df5)

svm_rbf_hasilsmote_1
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9746857896464474
Accuracy : 0.918904958677686
F-measure : 0.9078592977812351
Precision : 1.0
Recall : 0.9090909090909091
svm_rbf_hasilsmote_2
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9748604308446257
Accuracy : 0.9231663223140496
F-measure : 0.9114358089683635
Precision : 1.0
Recall : 0.9083161157024794
svm_rbf_hasilsmote_3
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9699539313202955
Accuracy : 0.9258780991735537
F-measure : 0.9145832117684626
Precision : 1.0
Recall : 0.9108987603305785
svm_rbf_hasilsmote_4
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9683368448861527
Accuracy : 0.921745867768595
F-measure : 0.911005668876379
Precision : 1.0
Recall : 0.9096074380165289
svm_rbf_hasilsmote_5
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9766685774405033
Ac

In [19]:

def prediksi(md,model,n,m,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data3/hasil_pred/{0}_{1}p_{2}_modelhasilsmote{3}.xlsx'.format(model.__name__,p,data.dt,d))

In [23]:
def fitn2pred(model,n,m,p,datalatih,datauji1,datauji2,datauji3,d):
    md=print_cv_results(model,n,m,datalatih)
    prediksi(md,model,n,m,p,datauji1,d)
    prediksi(md,model,n,m,p,datauji2,d)
    prediksi(md,model,n,m,p,datauji3,d)

In [25]:
#fitn2pred(mlp,100,1000,0.5,df1,dtf,dtl,dtg,1)